### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [2]:
import os
import oci
import time
import json
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta 

In [3]:
from oci.config import from_file
from oci.ai_anomaly_detection.models import *
from oci.ai_anomaly_detection.anomaly_detection_client import AnomalyDetectionClient

In [4]:
from oci.ai_anomaly_detection.models.create_project_details import CreateProjectDetails
from oci.ai_anomaly_detection.models.create_data_asset_details import CreateDataAssetDetails
from oci.ai_anomaly_detection.models.data_source_details import DataSourceDetails
from oci.ai_anomaly_detection.models.data_source_details_object_storage import DataSourceDetailsObjectStorage

In [5]:
from oci.ai_anomaly_detection.models.create_model_details import CreateModelDetails
from oci.ai_anomaly_detection.models.model_training_details import ModelTrainingDetails

In [6]:
from oci.ai_anomaly_detection.models.data_item import DataItem
from oci.ai_anomaly_detection.models.inline_detect_anomalies_request import InlineDetectAnomaliesRequest

In [7]:
#CONFIG_FILENAME = "/Users/USERNAME/.oci/config"
SERVICE_ENDPOINT="https://anomalydetection.aiservice.us-ashburn-1.oci.oraclecloud.com" 
NAMESPACE = "idkebl0fuyfa"
BUCKET_NAME = "bucket-ivy01-FAAS"
training_file_name="demo-training-data.csv"
compartment_id = "ocid1.compartment.oc1..aaaaaaaaaeys377iq4vzvrefmr6srfxt3tkaw4hhi3zd2nni2zm7umkjhuca"

In [8]:
#config = from_file(CONFIG_FILENAME)
config = oci.config.from_file('~/.oci/config')
ad_client = AnomalyDetectionClient(config,service_endpoint=SERVICE_ENDPOINT)

In [9]:
proj_details = CreateProjectDetails(display_name="Test-Project",description="Project description",compartment_id=compartment_id,)
create_res = ad_client.create_project(create_project_details=proj_details)
print("----CREATING----")
print(create_res.data)
time.sleep(5)
project_id = create_res.data.id

----CREATING----
{
  "compartment_id": "ocid1.compartment.oc1..aaaaaaaaaeys377iq4vzvrefmr6srfxt3tkaw4hhi3zd2nni2zm7umkjhuca",
  "defined_tags": {
    "Oracle-Tags": {
      "CreatedBy": "oracle/qian.jiang@oracle.com",
      "CreatedOn": "2022-09-21T09:17:39.356Z"
    }
  },
  "description": "Project description",
  "display_name": "Test-Project",
  "freeform_tags": {},
  "id": "ocid1.aianomalydetectionproject.oc1.iad.amaaaaaantfwtsqawknyzdjxu2hm4b4qrsxc4tcx5bmaqitbyksjmdkvf5vq",
  "lifecycle_state": "ACTIVE",
  "system_tags": {},
  "time_created": "2022-09-21T09:17:39.533000+00:00",
  "time_updated": "2022-09-21T09:17:39.533000+00:00"
}


In [10]:
get_proj = ad_client.get_project(project_id=project_id)
print("----READING---")
print(get_proj.data)

----READING---
{
  "compartment_id": "ocid1.compartment.oc1..aaaaaaaaaeys377iq4vzvrefmr6srfxt3tkaw4hhi3zd2nni2zm7umkjhuca",
  "defined_tags": {
    "Oracle-Tags": {
      "CreatedBy": "oracle/qian.jiang@oracle.com",
      "CreatedOn": "2022-09-21T09:17:39.356Z"
    }
  },
  "description": "Project description",
  "display_name": "Test-Project",
  "freeform_tags": {},
  "id": "ocid1.aianomalydetectionproject.oc1.iad.amaaaaaantfwtsqawknyzdjxu2hm4b4qrsxc4tcx5bmaqitbyksjmdkvf5vq",
  "lifecycle_state": "ACTIVE",
  "system_tags": {},
  "time_created": "2022-09-21T09:17:39.533000+00:00",
  "time_updated": "2022-09-21T09:17:39.533000+00:00"
}


In [11]:
list_proj = ad_client.list_projects(compartment_id=compartment_id)
print("----LISTING----")
print(list_proj.data)
time.sleep(5)

----LISTING----
{
  "items": [
    {
      "compartment_id": "ocid1.compartment.oc1..aaaaaaaaaeys377iq4vzvrefmr6srfxt3tkaw4hhi3zd2nni2zm7umkjhuca",
      "defined_tags": {
        "Oracle-Tags": {
          "CreatedBy": "oracle/qian.jiang@oracle.com",
          "CreatedOn": "2022-09-21T09:17:39.356Z"
        }
      },
      "description": "Project description",
      "display_name": "Test-Project",
      "freeform_tags": {},
      "id": "ocid1.aianomalydetectionproject.oc1.iad.amaaaaaantfwtsqawknyzdjxu2hm4b4qrsxc4tcx5bmaqitbyksjmdkvf5vq",
      "lifecycle_state": "ACTIVE",
      "system_tags": {},
      "time_created": "2022-09-21T09:17:39.533000+00:00",
      "time_updated": "2022-09-21T09:17:39.533000+00:00"
    },
    {
      "compartment_id": "ocid1.compartment.oc1..aaaaaaaaaeys377iq4vzvrefmr6srfxt3tkaw4hhi3zd2nni2zm7umkjhuca",
      "defined_tags": {
        "Oracle-Tags": {
          "CreatedBy": "oracle/qian.jiang@oracle.com",
          "CreatedOn": "2022-09-21T01:53:37.829Z"
 

In [12]:
print("-*-*-*-DATA ASSET-*-*-*-")

dDetails = DataSourceDetails(data_source_type="ORACLE_OBJECT_STORAGE")

dObjDeatils = DataSourceDetailsObjectStorage(
    namespace=NAMESPACE,
    bucket_name=BUCKET_NAME,
    object_name=training_file_name,
)

da_details = CreateDataAssetDetails(
    display_name="Test-DataAsset",
    description="DataAsset Description",
    compartment_id=compartment_id,
    project_id=project_id,
    data_source_details=dObjDeatils,
)
create_res = ad_client.create_data_asset(create_data_asset_details=da_details)
print("----CREATING----")
print(create_res.data)
da_id = create_res.data.id

-*-*-*-DATA ASSET-*-*-*-
----CREATING----
{
  "compartment_id": "ocid1.compartment.oc1..aaaaaaaaaeys377iq4vzvrefmr6srfxt3tkaw4hhi3zd2nni2zm7umkjhuca",
  "data_source_details": {
    "bucket_name": "bucket-ivy01-FAAS",
    "data_source_type": "ORACLE_OBJECT_STORAGE",
    "namespace": "idkebl0fuyfa",
    "object_name": "demo-training-data.csv"
  },
  "defined_tags": {
    "Oracle-Tags": {
      "CreatedBy": "oracle/qian.jiang@oracle.com",
      "CreatedOn": "2022-09-21T09:17:58.874Z"
    }
  },
  "description": "DataAsset Description",
  "display_name": "Test-DataAsset",
  "freeform_tags": {},
  "id": "ocid1.aianomalydetectiondataasset.oc1.iad.amaaaaaantfwtsqau2dbtlphgge6isdvakjyfxgp3r4kpnm2wtevthxfalja",
  "lifecycle_state": "ACTIVE",
  "private_endpoint_id": null,
  "project_id": "ocid1.aianomalydetectionproject.oc1.iad.amaaaaaantfwtsqawknyzdjxu2hm4b4qrsxc4tcx5bmaqitbyksjmdkvf5vq",
  "system_tags": {
    "orcl-cloud": {
      "free-tier-retained": "true"
    }
  },
  "time_created": "2

In [13]:
# READ CALL
get_da = ad_client.get_data_asset(data_asset_id=da_id)
print("----READING----")
print(get_da.data)

# LIST CALL
list_da = ad_client.list_data_assets(
    compartment_id=compartment_id, project_id=project_id
)
print("----LISTING----")
print(list_da.data)
time.sleep(5)

----READING----
{
  "compartment_id": "ocid1.compartment.oc1..aaaaaaaaaeys377iq4vzvrefmr6srfxt3tkaw4hhi3zd2nni2zm7umkjhuca",
  "data_source_details": {
    "bucket_name": "bucket-ivy01-FAAS",
    "data_source_type": "ORACLE_OBJECT_STORAGE",
    "namespace": "idkebl0fuyfa",
    "object_name": "demo-training-data.csv"
  },
  "defined_tags": {
    "Oracle-Tags": {
      "CreatedBy": "oracle/qian.jiang@oracle.com",
      "CreatedOn": "2022-09-21T09:17:58.874Z"
    }
  },
  "description": "DataAsset Description",
  "display_name": "Test-DataAsset",
  "freeform_tags": {},
  "id": "ocid1.aianomalydetectiondataasset.oc1.iad.amaaaaaantfwtsqau2dbtlphgge6isdvakjyfxgp3r4kpnm2wtevthxfalja",
  "lifecycle_state": "ACTIVE",
  "private_endpoint_id": null,
  "project_id": "ocid1.aianomalydetectionproject.oc1.iad.amaaaaaantfwtsqawknyzdjxu2hm4b4qrsxc4tcx5bmaqitbyksjmdkvf5vq",
  "system_tags": {
    "orcl-cloud": {
      "free-tier-retained": "true"
    }
  },
  "time_created": "2022-09-21T09:17:59.125000+

In [14]:
# MODEL
print("-*-*-*-MODEL-*-*-*-")
# CREATE CALL
dataAssetIds = [da_id]
mTrainDetails = ModelTrainingDetails(
    target_fap=0.02, training_fraction=0.7, data_asset_ids=dataAssetIds
)
mDetails = CreateModelDetails(
    display_name="Test-Model",
    description="Model description",
    compartment_id=compartment_id,
    project_id=project_id,
    model_training_details=mTrainDetails,
)

create_res = ad_client.create_model(create_model_details=mDetails)
print("----CREATING----")
print(create_res.data)
model_id = create_res.data.id

# READ CALL
get_model = ad_client.get_model(model_id=model_id)
print("----READING----")
print(get_model.data)
time.sleep(10)
while get_model.data.lifecycle_state == Model.LIFECYCLE_STATE_CREATING:
    get_model = ad_client.get_model(model_id=model_id)
    time.sleep(60)
    print(get_model.data.lifecycle_state)
print(get_model.data)

# LIST CALL
list_model = ad_client.list_models(compartment_id=compartment_id, project_id=project_id)
print("----LISTING----")
print(list_model.data)
time.sleep(10)

-*-*-*-MODEL-*-*-*-
----CREATING----
{
  "compartment_id": "ocid1.compartment.oc1..aaaaaaaaaeys377iq4vzvrefmr6srfxt3tkaw4hhi3zd2nni2zm7umkjhuca",
  "defined_tags": null,
  "description": "Model description",
  "display_name": "Test-Model",
  "freeform_tags": null,
  "id": "ocid1.aianomalydetectionmodel.oc1.iad.amaaaaaantfwtsqar656l53falng7wrlrn4phivu2q3yzsi7maasp5rkifqq",
  "lifecycle_details": "none",
  "lifecycle_state": "CREATING",
  "model_training_details": {
    "data_asset_ids": [
      "ocid1.aianomalydetectiondataasset.oc1.iad.amaaaaaantfwtsqau2dbtlphgge6isdvakjyfxgp3r4kpnm2wtevthxfalja"
    ],
    "target_fap": 0.02,
    "training_fraction": 0.7
  },
  "model_training_results": null,
  "project_id": "ocid1.aianomalydetectionproject.oc1.iad.amaaaaaantfwtsqawknyzdjxu2hm4b4qrsxc4tcx5bmaqitbyksjmdkvf5vq",
  "system_tags": null,
  "time_created": "2022-09-21T09:18:37.165000+00:00",
  "time_updated": "2022-09-21T09:18:37.165000+00:00"
}
----READING----
{
  "compartment_id": "ocid1.

In [21]:
# DETECT
print("-*-*-*-DETECT-*-*-*-")
## Method 1: Load the data from a csv file with first column as timestamp
# df = pd.read_csv(filename)
# signalNames = [e for e in df.columns if e != 'timestamp']

## Method 2: create a random dataframe with the appropriate header
num_rows = 1
signalNames = ["temperature_1", "temperature_2", "temperature_3", "temperature_4", "temperature_5", "pressure_1", "pressure_2", "pressure_3", "pressure_4", "pressure_5"]
# df = pd.DataFrame(np.random.rand(num_rows, len(signalNames)), columns=signalNames)
df = pd.DataFrame(data = [[-0.00000000000000001,-1.5588250424377889,-2.3941492028804823,2.593357112443483,-1.6094973523998164,-1.8641932916192756,-1.4677973361129089,-2.7708583572138727,-2.9587694388165655,1000000000000000]], columns=signalNames)
print(df)
#df.insert(0, 'timestamp', pd.date_range(start=date_today, periods=num_rows, freq='min')) # ID05042022.o
df.insert(0, 'timestamp', pd.date_range(start=date.today(), periods=num_rows, freq='min')) # ID05042022.n

#print(df)

df['timestamp'] = df['timestamp'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%SZ'))

# Now create the Payload from the dataframe
#print(df)


payloadData = []
for index, row in df.iterrows():
    timestamp = datetime.strptime(row['timestamp'], "%Y-%m-%dT%H:%M:%SZ")
    values = list(row[signalNames])
    dItem = DataItem(timestamp=timestamp, values=values)
    payloadData.append(dItem)
    


# ID05052022.so
#print("*********** Detect Payload ************");

#print("***************************************");
# ID05052022.eo


inline = InlineDetectAnomaliesRequest(model_id=model_id, request_type="INLINE", signal_names=signalNames, data=payloadData)






-*-*-*-DETECT-*-*-*-
   temperature_1  temperature_2  temperature_3  temperature_4  temperature_5  \
0  -1.000000e-17      -1.558825      -2.394149       2.593357      -1.609497   

   pressure_1  pressure_2  pressure_3  pressure_4        pressure_5  
0   -1.864193   -1.467797   -2.770858   -2.958769  1000000000000000  


In [22]:
detect_res = ad_client.detect_anomalies(detect_anomalies_details=inline)
print("----DETECTING----")
print(detect_res.data)

----DETECTING----
{
  "detection_results": [
    {
      "anomalies": [
        {
          "actual_value": 1000000000000000.0,
          "anomaly_score": 0.6,
          "estimated_value": -0.23813356517661172,
          "imputed_value": null,
          "signal_name": "pressure_5"
        }
      ],
      "row_index": null,
      "score": 0.20676177376184368,
      "timestamp": "2022-09-21T00:00:00+00:00"
    }
  ]
}


In [50]:

# DELETE MODEL
delete_model = ad_client.delete_model(model_id=model_id)
print("----DELETING MODEL----")
print(delete_model.data)
time.sleep(60)

# DELETE DATA ASSET
delete_da = ad_client.delete_data_asset(data_asset_id=da_id)
print("----DELETING DATA ASSET----")
print(delete_da.data)
time.sleep(10)

# DELETE PROJECT
print("----DELETING PROJECT----")
delete_project = ad_client.delete_project(project_id=project_id)
print(delete_project.data)

----DELETING MODEL----
None
----DELETING DATA ASSET----
None
----DELETING PROJECT----
None
